---
# Check if GPU is available

In [1]:
import torch
torch.cuda.is_available()

True

---
# Logger

In [2]:
from lightning.pytorch.loggers import NeptuneLogger
from dotenv import load_dotenv
import os

API_TOKEN = os.environ.get("LOGGER_API")
PROJECT_NAME = 'mrobay/Audio-project'

logger_config = {
    "api_key": API_TOKEN,
    "project_name": PROJECT_NAME,
    "log_model_checkpoints": False
}

---
# Loading Data

In [3]:
import importlib
imported_module = importlib.import_module("dataModule")
importlib.reload(imported_module)
from dataModule import AudioDataModule
dm = AudioDataModule()
dm.prepare_data()
dm.setup("fit")

# Create the classifier and train it

In [4]:
import importlib
importlib.reload(importlib.import_module("hebrewASR"))
importlib.reload(importlib.import_module("dataModule"))
from hebrewASR import HebrewASR, train_func, default_config
neptune_logger = NeptuneLogger(project=PROJECT_NAME, api_key=API_TOKEN, log_model_checkpoints=False)
trainer = train_func(config=default_config, logger=neptune_logger)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/nn/modules/rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/mrobay/Audio-project/e/AUD1-205


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | ctc_decoder  | GreedyCTCDecoder | 0      | train
1 | loss         | CTCLoss          | 0      | train
2 | conv_layers  | CNN              | 66.4 K | train
3 | bi_rnns      | ModuleList       | 9.2 M  | train
4 | linear_final | Linear           | 14.9 K | train
----------------------------------------------------------
9.3 M     Trainable params
0         Non-trainable para

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:122: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:122: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

# Validation

# Test the classifier

In [27]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
import classifier

trainer.test(model, data_loader['test'])

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

AttributeError: 'NeuralNetwork' object has no attribute 'argmax_prob'